In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd "/content/drive/My Drive/1_processing/w2v_cnn"

/content/drive/My Drive/1_processing/w2v_cnn


In [ ]:
import pandas as pd
import numpy as np
import pickle
from copy import deepcopy
import yaml

In [ ]:
with open('top10_list.pkl', 'rb') as input_file:
    top10_list = pickle.load(input_file)   
# top10_list

In [ ]:
with open('mutated.pkl', 'rb') as input_file:
    mutated = pickle.load(input_file)   
print(mutated.columns)
mutated.shape

Index(['repo_count', 'task_name', 'method_description', 'mod_keys_found',
       'mod_values_found', 'key_module_text', 'value_module_text',
       'mod_keys_found_string', 'found_used_parameters', 'has_params',
       'third_ast', 'third_tokens', 'token_task_names', 'descr_one_string',
       'consistent', 'task_com', 'task_complete'],
      dtype='object')


(20792, 17)

In [ ]:
mutated = mutated.drop(columns=['repo_count','mod_keys_found', 'mod_values_found', 'key_module_text', 'value_module_text','found_used_parameters', 'has_params','third_ast', 'third_tokens', 'token_task_names', 'descr_one_string','task_com'])

In [ ]:
mutated.to_excel('evaluation_set.xlsx')

In [ ]:
mutated.head(1)

repo_count  ...                                      task_complete
0           0  ...  [AnsibleTask, TaskName, restart, datadog-agent...

[1 rows x 17 columns]

In [ ]:
stf = pd.read_csv('ansible_data.csv')

In [ ]:
stf.head()

commit  ...                                 source_code_before
0  3ba18e654f145b24259521a6e650387466aec5fd  ...  #ansible-playbook jenkins.yml --extra-vars "ho...
1  6dcdc9946903493c318dbb0b1508222da8efbfbf  ...  #ansible-playbook jenkins.yml --extra-vars "ho...
2  2a08d7ff220ee623415878131afd8dd4e160ee18  ...  #ansible-playbook jenkins.yml --extra-vars "ho...
3  c2eabaaaf1d4985e23060b23538fa3ae6fb26dc9  ...  #ansible-playbook jenkins.yml --extra-vars "ho...
4  cabb8c12709d8ae5329f21200183b0821e8c93c1  ...  #ansible-playbook jenkins.yml --extra-vars "ho...

[5 rows x 7 columns]

In [ ]:
type(yaml.load(stf['source_code_after'][0]))

list

In [ ]:
stf2 = deepcopy(stf)

In [ ]:
def yaml_to_dict(text):

  task_list = []

  try:

    task_list = list(yaml.load(text))

  except:
    pass

  return task_list

In [ ]:
stf2['tasks_list_dicts'] = stf2['source_code_before'].apply(lambda x: yaml_to_dict(x))

In [ ]:
stf2.shape

(3387, 8)

In [ ]:
stf3 = deepcopy(stf2)

In [ ]:
stf3 = stf3[stf3['tasks_list_dicts'].map(lambda d: len(d)) > 0]
stf3.shape

(3366, 8)

In [ ]:
tasks_lst = []
for tsks in stf3['tasks_list_dicts']:
  for tsk in tsks:
     tasks_lst.append(tsk)

In [ ]:
tasks_lst[1700].keys()

dict_keys(['block', 'when'])

In [ ]:
for t in tasks_lst:
  if type(t) != dict:
    print(t)

named_config_views
ansible_user
ansible_become
osp_volumes
osp_security_groups
osp_instances


In [ ]:
len(tasks_lst)

31699

In [ ]:
tasks_lst = [tsk for tsk in tasks_lst if type(tsk) == dict ]

In [ ]:
len(tasks_lst)

31693

In [ ]:
set(list(tasks_lst[19].keys()))

{'file', 'name'}

In [ ]:
if list(set(list(tasks_lst[19].keys())) & set(top10_list)):
  print('yes')

yes


In [ ]:
top_stf_tsk_list = [tsk for tsk in tasks_lst if list(set(tsk.keys()) & set(top10_list))]

In [ ]:
len(top_stf_tsk_list)

16928

In [ ]:
top_stf_tsk_list[0]

{'copy': 'src=download-jdk7.sh dest=/tmp mode=0555',
 'name': 'Copy download JDK7 script'}

In [ ]:
# dftsk_stf = pd.DataFrame(top_stf_tsk_list, columns =['tasks'])

In [ ]:
mted = deepcopy(mutated.head(10396))

In [ ]:
m_tsk_names = mted['task_name'].tolist()

In [ ]:
# Remove tasks found in both sets

In [ ]:
clean_list = []
count = 0 
for t in top_stf_tsk_list:

  try:
    if t['name'] not in m_tsk_names:
      clean_list.append(t)
      count += 1

  except:
    pass

In [ ]:
len(clean_list)

14187

In [ ]:
clean_list[879]

{'file': {'path': '/tmp/linklight', 'state': 'absent'},
 'name': 'clean out /tmp/linklight'}

In [ ]:
clean_tasks_df = pd.DataFrame(columns=['task_name','method_description'])

In [ ]:
count = 0
for tskk in clean_list:
  
  task_names = {}
  task_body = {}

  task_names.update(name=tskk['name'])
  body_list = []

  for key,value in tskk.items():
    if key != 'name':
      task_body.update([(key,value)])
  
  body_list.append(task_body)
  print(body_list)

  clean_tasks_df = pd.concat([clean_tasks_df, pd.DataFrame({'task_name':task_names['name'],'method_description':body_list}, index=[0])], ignore_index = True)
  count += 1
  # if count == 1000:
  #   break

Streaming output truncated to the last 5000 lines.
[{'template': {'src': 'var2.yml', 'dest': '/opt/appdata/plexguide/var2.yml', 'force': True}}]
[{'shell': "hostname -I | awk '{print $1}'", 'register': 'ip_address'}]
[{'shell': "hostname -I | awk '{print $1}' | sed 's/[0-9]*$/0/'", 'register': 'lan_net'}]
[{'file': 'path={{item}} state=directory mode=0755 owner=1000 group=1000', 'with_items': ['/opt/appdata/vpn', '/opt/appdata/vpn/deluge', '/opt/appdata/vpn/deluge/config', '/opt/appdata/vpn/deluge/config/openvpn']}]
[{'template': {'src': 'netherlands.ovpn', 'dest': '/opt/appdata/vpn/deluge/config/openvpn/netherlands.ovpn', 'force': True}}]
[{'shell': 'rm -r  /opt/appdata/vpn/deluge/config/openvpn/*.ovpn'}]
[{'template': {'src': '{{server.user_input}}.ovpn', 'dest': '/opt/appdata/vpn/deluge/config/openvpn/{{server.user_input}}.ovpn', 'force': True}}]
[{'file': 'path={{item}} state=directory mode=0755 owner=1000 group=1000', 'with_items': ['/opt/appdata/vpn', '/opt/appdata/vpn/rtorrent',

In [ ]:
clean_tasks_df = clean_tasks_df.reset_index(drop=True)

In [ ]:
clean_tasks_df['method_description'][3]

{'file': 'path=/opt/maven321 src=/opt/apache-maven-3.2.1 state=link'}

In [ ]:
print(clean_tasks_df['task_name'][0])
clean_tasks_df['method_description'][0]

Copy download JDK7 script


{'copy': 'src=download-jdk7.sh dest=/tmp mode=0555'}

In [ ]:
clean_list[0]

{'copy': 'src=download-jdk7.sh dest=/tmp mode=0555',
 'name': 'Copy download JDK7 script'}

In [ ]:
with open('clean_stefano_tasks.pkl', 'wb') as output_file:
    pickle.dump(clean_tasks_df, output_file)

In [ ]:
with open('clean_stefano_tasks.pkl', 'rb') as input_file:
    mapped10_stef_tasks = pickle.load(input_file)   
mapped10_stef_tasks.shape

(14187, 2)

In [ ]:
print(mapped10_stef_tasks['task_name'][0])
mapped10_stef_tasks['method_description'][0]['copy']

Copy download JDK7 script


'src=download-jdk7.sh dest=/tmp mode=0555'

In [ ]:
from ansible_modules import get_ansible_modules,top10_modules_used
from ident_ans_mods import identify_ansible_modules
from collections import Counter
from  itertools import chain
%load_ext autoreload
%autoreload 2

In [ ]:
descriptions = mapped10_stef_tasks[['task_name','method_description']]

In [ ]:
mda = get_ansible_modules('https://docs.ansible.com/ansible/2.8/modules/list_of_all_modules.html')

In [ ]:
# mda

In [ ]:
df_desc_mod = descriptions.copy()
df_desc_mod = df_desc_mod.reset_index()
df_desc_mod = df_desc_mod.rename(columns={'index':'repo_count'})
# df_desc_mod

In [ ]:
inst = identify_ansible_modules(df_desc_mod,mda)

In [ ]:
df = inst.create_text_cols()

In [ ]:
df.head()

repo_count  ... value_module_text
0           0  ...                  
1           1  ...                  
2           2  ...                  
3           3  ...                  
4           4  ...                  

[5 rows x 7 columns]

In [ ]:
df[df['mod_keys_found'].apply(len).gt(0)]

repo_count  ...     value_module_text
0               0  ...                      
1               1  ...                      
2               2  ...                      
3               3  ...                      
4               4  ...                      
...           ...  ...                   ...
14182       14182  ...    {{ sshd_service }}
14183       14183  ...  Add or remove groups
14184       14184  ...    {{ sshd_service }}
14185       14185  ...                      
14186       14186  ...    {{ sshd_service }}

[14187 rows x 7 columns]

In [ ]:
df = df[df['mod_keys_found'].apply(len).gt(0)]

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
tasks_ast_raw = deepcopy(df)

In [ ]:
tasks_ast_raw = tasks_ast_raw[tasks_ast_raw['mod_keys_found'].apply(lambda v: len(set(v) & set(top10_list)) > 0)]

In [ ]:
tasks_ast_raw.shape

(14187, 7)

In [ ]:
tasks_10 = tasks_ast_raw.copy()

In [ ]:
one_key_set = tasks_10[tasks_10['mod_keys_found'].apply(lambda x: len(x) < 2)]
one_key_set.shape

(14116, 7)

In [ ]:
one_key_set = one_key_set.reset_index(drop=True)

In [ ]:
counted_modules, counted_modules_df = top10_modules_used(one_key_set)
counted_modules

['file',
 'shell',
 'set_fact',
 'command',
 'template',
 'copy',
 'service',
 'fail',
 'debug',
 'gather_facts']

In [ ]:
counted_modules_df.head(10)

ansible_modules  frequency
4            file       2617
8           shell       2297
7        set_fact       2236
0         command       2200
9        template       2025
1            copy       1111
6         service        791
3            fail        503
2           debug        245
5    gather_facts         91

In [ ]:
import sys
sys.path.insert(0,"/content/drive/My Drive/1_processing/w2v_cnn")

In [ ]:
!pip install treelib

  Created wheel for treelib: filename=treelib-1.6.1-py3-none-any.whl size=18386 sha256=17cbe58207c742e93b67f752cf87435bdfd688267fbed273185a385abf2f51c9
  Stored in directory: /root/.cache/pip/wheels/89/be/94/2c6d949ce599d1443426d83ba4dc93cd35c0f4638260930a53
Successfully built treelib


In [ ]:
!pip install anytree

     |████████████████████████████████| 41 kB 397 kB/s 


In [ ]:
from build_sequence_tokens import return_tokens, rem_nested_lists,convert_int_bool_to_str,tokenize_task,tokenize_name,is_empty

In [ ]:
one_key_set['initial_ast'] = one_key_set['method_description'].apply(lambda x: return_tokens(x))
one_key_set['second_ast'] = one_key_set['initial_ast'].apply(lambda x: convert_int_bool_to_str(x))
one_key_set['third_ast'] = one_key_set['second_ast'].apply(lambda x: rem_nested_lists(x))

In [ ]:
one_key_set['method_description'][0]

{'copy': 'src=download-jdk7.sh dest=/tmp mode=0555'}

In [ ]:
import nltk
from nltk import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
one_key_set = one_key_set.reset_index(drop=True)

In [ ]:
task_name_tokens = []
except_list = []
for idx, tname in one_key_set['task_name'].items():
  try:
    task_name_tokens.append(word_tokenize(tname))
  except:
    except_list.append(idx)
    task_name_tokens.append('')

In [ ]:
one_key_set['token_task_names'] = task_name_tokens

In [ ]:
except_list

[11665, 11668, 11669]

In [ ]:
type(one_key_set['token_task_names'][11665])

str

In [ ]:
one_key_set.shape

(14116, 11)

In [ ]:
one_key_set = one_key_set[one_key_set['token_task_names'] != '']
one_key_set.shape

(14113, 11)

In [ ]:
one_key_set['token_task_names'][0]

['Copy', 'download', 'JDK7', 'script']

In [ ]:
mapped10 = deepcopy(one_key_set)

In [ ]:
mapped10['first_tokens'] = mapped10['third_ast'].apply(lambda x: tokenize_task(x))

In [ ]:
mapped10 = mapped10.reset_index(drop=True)

In [ ]:
m10 = deepcopy(mapped10)

In [ ]:
from pre_process import remove_symbols, remove_symbols_simple,flatten_list

In [ ]:
m10['second_tokens'] = m10['first_tokens'].apply(lambda x: remove_symbols(x))
m10['third_tokens'] = m10['second_tokens'].apply(lambda x: flatten_list(x))

In [ ]:
m10 = m10.drop(columns=['initial_ast', 'second_ast','first_tokens','second_tokens'])

In [ ]:
m10['descr_one_string'] = m10['third_tokens'].apply(lambda x: ' '.join(x))

In [ ]:
remove_list = [',', '``','"',"''",'==','{','}','(',"'",'[',']',"'==",')','>','=','-','|','/','+','$','.+',':','!',
               '--','\\','?','@',';','&','#','^','<','*','%','.','=~','{{','}}',]

def remove_symbols_simple(sequence):
  for item in sequence:
    if item in remove_list:
      sequence.remove(item)

  return sequence

In [ ]:
m20 = deepcopy(m10)

In [ ]:
m20['token_names_one_string'] = m20['token_task_names'].apply(lambda x: remove_symbols_simple(x))
m20['third_tokens'] = m20['third_tokens'].apply(lambda x: remove_symbols_simple(x))

In [ ]:
m = deepcopy(m20)

In [ ]:
m['token_task_names'] = m['token_task_names'].apply(lambda x: ['TaskName']+x)

In [ ]:
def correct_first_token_name(seq):
  seq[0] = 'TaskDescription'
  return seq

In [ ]:
mm = deepcopy(m)

In [ ]:
mm['third_tokens'] = mm['third_tokens'].apply(lambda x: correct_first_token_name(x))

In [ ]:
mm['token_task_names'][0]

['TaskName', 'Copy', 'download', 'JDK7', 'script']

In [ ]:
merged2 = deepcopy(mm)

In [ ]:
merged2['task_com'] = merged2['token_task_names']+merged2['third_tokens']

In [ ]:
def correct_first_token_name2(seq):
  seq.insert(0,'AnsibleTask')
  return seq

In [ ]:
merged2['task_complete'] = merged2['task_com'].apply(lambda x: correct_first_token_name2(x))

In [ ]:
merged2['task_complete'][0]

['AnsibleTask',
 'TaskName',
 'Copy',
 'download',
 'JDK7',
 'script',
 'TaskDescription',
 'copy',
 'src=download-jdk7.sh',
 'dest=/tmp',
 'mode=0555']

In [ ]:
merged2.to_pickle('val_set.pkl')

In [ ]:
with open('val_set.pkl', 'rb') as input_file:
    val_set = pickle.load(input_file)   
val_set.columns

Index(['repo_count', 'task_name', 'method_description', 'mod_keys_found',
       'mod_values_found', 'key_module_text', 'value_module_text', 'third_ast',
       'token_task_names', 'third_tokens', 'descr_one_string',
       'token_names_one_string', 'task_com', 'task_complete'],
      dtype='object')

In [ ]:
val_set.shape

(13754, 14)